<a href="https://colab.research.google.com/github/CodeHunterOfficial/ABC_DataMining/blob/main/NM/Creating_a_new_korpus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install python-docx langdetect PyPDF2 bs4 langdetect readability

  Preparing metadata (setup.py) ... done
  Created wheel for readability: filename=readability-0.3.2-py3-none-any.whl size=36384 sha256=d97401060a24f14c2c11ce940aa8c8de24876048f903a7eea101d75ced4a1347
  Stored in directory: /root/.cache/pip/wheels/6a/a8/01/0b6587e224d9731dae317fdad11b081f0e8b7be7d8367fc6eb
Successfully built readability


In [ ]:
import os
import re
from docx import Document
import logging
from typing import List, Dict, Optional
import nltk  # Для разделения на предложения
from langdetect import detect  # Для определения языка
import json  # Для сохранения в JSON
import xml.etree.ElementTree as ET  # Для сохранения в XML
from PyPDF2 import PdfReader  # Для чтения PDF
from bs4 import BeautifulSoup  # Для парсинга HTML

# Настройка логирования
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")


class BookCorpusProcessor:
    def __init__(self, books_folder: str, output_base: str = "tajik_books"):
        """
        Инициализация класса.

        :param books_folder: Путь к папке с книгами.
        :param output_base: Базовое имя выходных файлов (без расширений).
        """
        self.books_folder = books_folder
        self.output_base = output_base
        self.processed_books: List[str] = []

    def clean_text(self, text: str, custom_patterns: Optional[List[str]] = None) -> str:
        """
        Очищает текст от лишних символов и номеров страниц.

        :param text: Исходный текст.
        :param custom_patterns: Список пользовательских регулярных выражений для очистки текста.
        :return: Очищенный текст.
        """
        patterns = [
            r"^\s*\d+\s*$",  # Удалить номера страниц
            r"[^\w\s\.,!?;:()«»“”'\"\\/-]",  # Удалить специальные символы
            r"\s+",  # Заменить множественные пробелы на один пробел
            r"^\s*$",  # Удалить пустые строки
            r"\t+",  # Удалить табуляции
            r"\.{2,}",  # Заменить многоточия на точку
            r"<[^>]*>",  # Удалить HTML-теги
        ]

        if custom_patterns:
            patterns.extend(custom_patterns)

        for pattern in patterns:
            text = re.sub(pattern, " ", text, flags=re.MULTILINE)

        return text.strip()

    def extract_metadata(self, filename: str) -> Dict[str, str]:
        """
        Извлекает метаданные (название и автора) из имени файла.

        :param filename: Имя файла.
        :return: Словарь с метаданными.
        """
        base_name = os.path.splitext(filename)[0]
        parts = base_name.split("_", 1)

        if len(parts) == 2:
            title, author = parts
            return {"title": title.strip(), "author": author.strip()}
        else:
            return {"title": base_name.strip(), "author": "Неизвестный"}

    def process_docx_file(self, file_path: str) -> str:
        """
        Обрабатывает один .docx файл, извлекает текст и метаданные.

        :param file_path: Путь к файлу.
        :return: Обработанный текст с метаданными или пустая строка при ошибке.
        """
        try:
            doc = Document(file_path)
            paragraphs = [paragraph.text for paragraph in doc.paragraphs]
            raw_text = "\n".join(paragraphs)
            cleaned_text = self.clean_text(raw_text)
            metadata = self.extract_metadata(os.path.basename(file_path))
            metadata_str = f"# Название: {metadata['title']}\n# Автор: {metadata['author']}\n# -----\n"
            return metadata_str + cleaned_text + "\n\n"
        except Exception as e:
            logging.error(f"Ошибка при обработке файла {file_path}: {e}")
            return ""

    def process_txt_file(self, file_path: str) -> str:
        """
        Обрабатывает один .txt файл, извлекает текст и метаданные.

        :param file_path: Путь к файлу.
        :return: Обработанный текст с метаданными или пустая строка при ошибке.
        """
        try:
            with open(file_path, "r", encoding="utf-8") as file:
                raw_text = file.read()
            cleaned_text = self.clean_text(raw_text)
            metadata = self.extract_metadata(os.path.basename(file_path))
            metadata_str = f"# Название: {metadata['title']}\n# Автор: {metadata['author']}\n# -----\n"
            return metadata_str + cleaned_text + "\n\n"
        except Exception as e:
            logging.error(f"Ошибка при обработке файла {file_path}: {e}")
            return ""

    def process_pdf_file(self, file_path: str) -> str:
        """
        Обрабатывает один .pdf файл, извлекает текст и метаданные.

        :param file_path: Путь к файлу.
        :return: Обработанный текст с метаданными или пустая строка при ошибке.
        """
        try:
            reader = PdfReader(file_path)
            raw_text = "\n".join([page.extract_text() for page in reader.pages])
            cleaned_text = self.clean_text(raw_text)
            metadata = self.extract_metadata(os.path.basename(file_path))
            metadata_str = f"# Название: {metadata['title']}\n# Автор: {metadata['author']}\n# -----\n"
            return metadata_str + cleaned_text + "\n\n"
        except Exception as e:
            logging.error(f"Ошибка при обработке файла {file_path}: {e}")
            return ""

    def process_html_file(self, file_path: str) -> str:
        """
        Обрабатывает один .html файл, извлекает текст и метаданные.

        :param file_path: Путь к файлу.
        :return: Обработанный текст с метаданными или пустая строка при ошибке.
        """
        try:
            with open(file_path, "r", encoding="utf-8") as file:
                soup = BeautifulSoup(file.read(), 'html.parser')

            # Удаляем скрипты и стили
            for script_or_style in soup(["script", "style"]):
                script_or_style.decompose()

            raw_text = soup.get_text(separator="\n")
            cleaned_text = self.clean_text(raw_text)
            metadata = self.extract_metadata(os.path.basename(file_path))
            metadata_str = f"# Название: {metadata['title']}\n# Автор: {metadata['author']}\n# -----\n"
            return metadata_str + cleaned_text + "\n\n"
        except Exception as e:
            logging.error(f"Ошибка при обработке файла {file_path}: {e}")
            return ""

    def process_all_books(self):
        """
        Обрабатывает все файлы в указанной папке и сохраняет результат.
        """
        if not os.path.exists(self.books_folder):
            logging.error("Указанный путь не существует.")
            return

        all_books = []
        for filename in os.listdir(self.books_folder):
            file_path = os.path.join(self.books_folder, filename)

            if filename.endswith(".docx"):
                processed_text = self.process_docx_file(file_path)
            elif filename.endswith(".txt"):
                processed_text = self.process_txt_file(file_path)
            elif filename.endswith(".pdf"):
                processed_text = self.process_pdf_file(file_path)
            elif filename.endswith(".html"):
                processed_text = self.process_html_file(file_path)
            else:
                logging.warning(f"Файл {filename} пропущен из-за неподдерживаемого формата.")
                continue

            if processed_text:
                all_books.append(processed_text)
                self.processed_books.append(filename)
                logging.info(f"Обработан файл: {filename}")
            else:
                logging.warning(f"Файл {filename} пропущен из-за ошибки.")

        # Сохранение в TXT
        with open(f"{self.output_base}.txt", "w", encoding="utf-8") as txt_file:
            txt_file.write("\n".join(all_books))

        # Сохранение в JSON
        json_data = []
        for book in all_books:
            title, rest = book.split("\n", 1)
            author, text = rest.split("# -----\n", 1)
            json_data.append({
                "title": title.split(":")[1].strip(),
                "author": author.split(":")[1].strip(),
                "text": text.strip()
            })
        with open(f"{self.output_base}.json", "w", encoding="utf-8") as json_file:
            json.dump(json_data, json_file, ensure_ascii=False, indent=4)

        # Сохранение в XML
        root = ET.Element("books")
        for book in all_books:
            title, rest = book.split("\n", 1)
            author, text = rest.split("# -----\n", 1)
            book_elem = ET.SubElement(root, "book")
            ET.SubElement(book_elem, "title").text = title.split(":")[1].strip()
            ET.SubElement(book_elem, "author").text = author.split(":")[1].strip()
            ET.SubElement(book_elem, "text").text = text.strip()

        tree = ET.ElementTree(root)
        tree.write(f"{self.output_base}.xml", encoding="utf-8", xml_declaration=True)

        logging.info(f"Все книги успешно объединены в {self.output_base}.txt, {self.output_base}.json и {self.output_base}.xml.")
        logging.info(f"Обработано книг: {len(self.processed_books)}.")

    def split_into_sentences(self, text: str) -> str:
        """
        Разделяет текст на предложения.

        :param text: Исходный текст.
        :return: Текст, разделенный на предложения.
        """
        try:
            nltk.download('punkt', quiet=True)
            sentences = nltk.sent_tokenize(text, language="russian")
            return "\n".join(sentences)
        except Exception as e:
            logging.warning(f"Ошибка при разделении текста на предложения: {e}")
            return text

    def detect_language(self, text: str) -> str:
        """
        Определяет язык текста.

        :param text: Исходный текст.
        :return: Язык текста (например, 'ru', 'tg', 'en').
        """
        try:
            return detect(text[:1000])  # Проверяем первые 1000 символов
        except:
            return "unknown"


# Пример использования класса
if __name__ == "__main__":
    # Путь к папке с книгами
    books_folder = input("Введите путь к папке с книгами: ").strip()

    # Создание экземпляра класса
    processor = BookCorpusProcessor(books_folder, output_base="tajik_books")

    # Обработка всех книг
    processor.process_all_books()

Введите путь к папке с книгами: /content/


# CleanCorpusProcessor

In [ ]:
import os
import logging
import json
from typing import List, Dict
import nltk
from langdetect import detect
from transformers import pipeline
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import xml.etree.ElementTree as ET

# Настройка логирования
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

# Скачиваем необходимые ресурсы NLTK
nltk.download('punkt', quiet=True)
nltk.download('stopwords', quiet=True)
nltk.download('wordnet', quiet=True)
nltk.download('punkt_tab')

class CleanCorpusProcessor:
    def __init__(self, input_file: str, output_base: str = "clean_corpus"):
        """
        Инициализация класса.
        :param input_file: Путь к файлу (JSON, TXT или XML), созданному BookCorpusProcessor.
        :param output_base: Базовое имя выходных файлов.
        """
        self.input_file = input_file
        self.output_base = output_base
        self.stop_words = set(stopwords.words('russian') + stopwords.words('english'))
        self.spell_correction_model = pipeline("text2text-generation", model="google/flan-t5-large")
        self.lemmatizer = WordNetLemmatizer()
        self.processed_books: List[Dict] = []

    def load_data(self) -> List[Dict]:
        """
        Загружает данные из файла (JSON, TXT или XML).
        """
        if not os.path.exists(self.input_file):
            logging.error(f"Файл {self.input_file} не найден.")
            return []

        _, ext = os.path.splitext(self.input_file)
        if ext == ".json":
            return self.load_from_json()
        elif ext == ".txt":
            return self.load_from_txt()
        elif ext == ".xml":
            return self.load_from_xml()
        else:
            logging.error(f"Неподдерживаемый формат файла: {ext}")
            return []

    def load_from_json(self) -> List[Dict]:
        """
        Загружает данные из JSON-файла.
        """
        with open(self.input_file, "r", encoding="utf-8") as file:
            data = json.load(file)
        return data

    def load_from_txt(self) -> List[Dict]:
        """
        Загружает данные из TXT-файла.
        """
        books = []
        with open(self.input_file, "r", encoding="utf-8") as file:
            content = file.read().strip().split("\n\n")
        for book in content:
            try:
                title, rest = book.split("\n", 1)
                author, text = rest.split("# -----\n", 1)
                books.append({
                    "title": title.split(":")[1].strip(),
                    "author": author.split(":")[1].strip(),
                    "text": text.strip()
                })
            except Exception as e:
                logging.warning(f"Ошибка при чтении книги из TXT: {e}")
        return books

    def load_from_xml(self) -> List[Dict]:
        """
        Загружает данные из XML-файла.
        """
        books = []
        tree = ET.parse(self.input_file)
        root = tree.getroot()
        for book_elem in root.findall("book"):
            title = book_elem.find("title").text.strip() if book_elem.find("title") is not None else "Unknown Title"
            author = book_elem.find("author").text.strip() if book_elem.find("author") is not None else "Unknown Author"
            text = book_elem.find("text").text.strip() if book_elem.find("text") is not None else ""
            books.append({"title": title, "author": author, "text": text})
        return books

    def correct_spelling(self, text: str) -> str:
        """
        Исправляет опечатки в тексте.
        """
        try:
            sentences = nltk.sent_tokenize(text)
            corrected_sentences = [self.spell_correction_model(sentence)[0]['generated_text'] for sentence in sentences]
            return " ".join(corrected_sentences)
        except Exception as e:
            logging.warning(f"Ошибка при исправлении опечаток: {e}")
            return text

    def lemmatize_text(self, text: str) -> str:
        """
        Лемматизирует текст.
        """
        tokens = word_tokenize(text.lower())
        lemmatized_tokens = [self.lemmatizer.lemmatize(token) for token in tokens]
        return " ".join(lemmatized_tokens)

    def remove_stopwords(self, text: str) -> str:
        """
        Удаляет стоп-слова из текста.
        """
        tokens = word_tokenize(text.lower())
        filtered_tokens = [token for token in tokens if token not in self.stop_words]
        return " ".join(filtered_tokens)

    def process_book(self, book: Dict) -> Dict:
        """
        Обрабатывает одну книгу: исправляет опечатки, лемматизирует и удаляет стоп-слова.
        """
        title = book.get("title", "Unknown Title")
        author = book.get("author", "Unknown Author")
        raw_text = book.get("text", "")

        # Исправление опечаток
        corrected_text = self.correct_spelling(raw_text)

        # Лемматизация
        lemmatized_text = self.lemmatize_text(corrected_text)

        # Удаление стоп-слов
        clean_text = self.remove_stopwords(lemmatized_text)

        return {
            "title": title,
            "author": author,
            "language": detect(raw_text[:1000]) if raw_text else "unknown",
            "text": clean_text
        }

    def process_all_books(self):
        """
        Обрабатывает все книги из входного файла и сохраняет результат.
        """
        books_data = self.load_data()
        if not books_data:
            logging.error("Нет данных для обработки.")
            return

        for book in books_data:
            processed_book = self.process_book(book)
            if processed_book:
                self.processed_books.append(processed_book)
                logging.info(f"Обработана книга: {processed_book['title']}")

        # Сохранение в JSON
        with open(f"{self.output_base}.json", "w", encoding="utf-8") as json_file:
            json.dump(self.processed_books, json_file, ensure_ascii=False, indent=4)

        # Сохранение в XML
        root = ET.Element("books")
        for book in self.processed_books:
            book_elem = ET.SubElement(root, "book")
            ET.SubElement(book_elem, "title").text = book["title"]
            ET.SubElement(book_elem, "author").text = book["author"]
            ET.SubElement(book_elem, "language").text = book["language"]
            ET.SubElement(book_elem, "text").text = book["text"]
        tree = ET.ElementTree(root)
        tree.write(f"{self.output_base}.xml", encoding="utf-8", xml_declaration=True)

        logging.info(f"Все книги успешно обработаны и сохранены в {self.output_base}.json и {self.output_base}.xml.")
        logging.info(f"Обработано книг: {len(self.processed_books)}.")

# Пример использования класса
if __name__ == "__main__":
    input_file = input("Введите путь к файлу (JSON, TXT или XML): ").strip()
    processor = CleanCorpusProcessor(input_file, output_base="clean_corpus")
    processor.process_all_books()

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


Введите путь к файлу (JSON, TXT или XML): /content/tajik_books.txt


Device set to use cpu


#Web

In [1]:
!pip install pytube tweepy readability-lxml lxml_html_clean

In [4]:
import os
import re
import logging
from typing import List, Dict, Optional  # Добавлен импорт Optional
import json
import requests
from bs4 import BeautifulSoup
from readability import Document  # Для извлечения основного текста
import xml.etree.ElementTree as ET  # Для создания XML

# Настройка логирования
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

class WebCorpusProcessor:
    def __init__(self, output_base: str = "news_corpus"):
        """
        Инициализация класса.
        :param output_base: Базовое имя выходных файлов (без расширений).
        """
        self.output_base = output_base
        self.processed_items: List[Dict] = []

    def clean_text(self, text: str, custom_patterns: Optional[List[str]] = None) -> str:
        """
        Очищает текст от HTML-тегов и лишних символов.
        :param text: Исходный текст.
        :param custom_patterns: Список пользовательских регулярных выражений для очистки текста.
        :return: Очищенный текст.
        """
        soup = BeautifulSoup(text, 'html.parser')
        plain_text = soup.get_text(separator=" ")
        patterns = [
            r"^\s*\d+\s*$",  # Удалить номера страниц
            r"[^\w\s\.,!?;:()«»“”'\"\\/-]",  # Удалить специальные символы
            r"\s+",  # Заменить множественные пробелы на один пробел
            r"^\s*$",  # Удалить пустые строки
            r"\t+",  # Удалить табуляции
            r"\.{2,}",  # Заменить многоточия на точку
        ]
        if custom_patterns:
            patterns.extend(custom_patterns)
        for pattern in patterns:
            plain_text = re.sub(pattern, " ", plain_text, flags=re.MULTILINE)
        return plain_text.strip()

    def clean_content(self, data: Dict) -> Dict:
        """
        Очищает содержимое словаря с ключами 'title', 'author' и 'content'.
        :param data: Словарь с данными.
        :return: Очищенный словарь.
        """
        cleaned_data = {
            "title": self.clean_text(data.get("title", "")),
            "author": self.clean_text(data.get("author", "")),
            "content": self.clean_text(data.get("content", ""))
        }
        return cleaned_data

    def extract_web_content(self, url: str) -> Dict:
        """
        Извлекает заголовок, автора и основной текстовый контент из веб-страницы.
        :param url: URL веб-страницы.
        :return: Словарь с заголовком, автором и текстом.
        """
        try:
            response = requests.get(url, timeout=10)
            response.raise_for_status()
            soup = BeautifulSoup(response.text, 'html.parser')
            title = soup.title.string.strip() if soup.title else "No Title"
            author_tag = soup.find("meta", attrs={"name": "author"})
            author = author_tag["content"].strip() if author_tag else "Unknown Author"
            doc = Document(response.text)
            raw_text = doc.summary()
            raw_data = {
                "title": title,
                "author": author,
                "content": raw_text
            }
            cleaned_data = self.clean_content(raw_data)
            return cleaned_data
        except Exception as e:
            logging.error(f"Ошибка при извлечении контента из {url}: {e}")
            return {"title": "Error", "author": "Unknown", "content": ""}

    def process_all_sources(self, sources: List[Dict]):
        """
        Обрабатывает все источники данных (только веб-сайты).
        :param sources: Список источников данных.
        """
        all_data = []
        for source in sources:
            if source["type"] == "web":
                content = self.extract_web_content(source["url"])
                if content:
                    all_data.append({"source": "web", "url": source["url"], "content": content})

        # Сохранение в JSON
        self.save_to_json(all_data)
        # Сохранение в TXT
        self.save_to_txt(all_data)
        # Сохранение в XML
        self.save_to_xml(all_data)
        logging.info(f"Все данные успешно сохранены в {self.output_base}.json, {self.output_base}.txt и {self.output_base}.xml.")
        logging.info(f"Обработано источников: {len(all_data)}.")

    def save_to_json(self, data: List[Dict]):
        """
        Сохраняет данные в JSON-файл.
        :param data: Список словарей с данными.
        """
        with open(f"{self.output_base}.json", "w", encoding="utf-8") as json_file:
            json.dump(data, json_file, ensure_ascii=False, indent=4)

    def save_to_txt(self, data: List[Dict]):
        """
        Сохраняет данные в TXT-файл.
        :param data: Список словарей с данными.
        """
        with open(f"{self.output_base}.txt", "w", encoding="utf-8") as txt_file:
            for item in data:
                txt_file.write(f"Title: {item.get('content', {}).get('title', 'N/A')}\n")
                txt_file.write(f"Author: {item.get('content', {}).get('author', 'N/A')}\n")
                txt_file.write(f"Content: {item.get('content', {}).get('content', '')}\n\n")

    def save_to_xml(self, data: List[Dict]):
        """
        Сохраняет данные в XML-файл.
        :param data: Список словарей с данными.
        """
        root = ET.Element("news_corpus")
        for item in data:
            entry = ET.SubElement(root, "entry")
            content = item.get("content", {})
            ET.SubElement(entry, "title").text = content.get("title", "N/A")
            ET.SubElement(entry, "author").text = content.get("author", "N/A")
            ET.SubElement(entry, "content").text = content.get("content", "")
        tree = ET.ElementTree(root)
        tree.write(f"{self.output_base}.xml", encoding="utf-8", xml_declaration=True)

# Пример использования класса
if __name__ == "__main__":
    # Создание экземпляра класса
    processor = WebCorpusProcessor(output_base="web_news")
    # Список источников данных (только веб-сайты)
    sources = [
        {"type": "web", "url": "https://en.wikipedia.org/wiki/Web_scraping"},
        {"type": "web", "url": "https://docs.python.org/3/tutorial/index.html"},
        {"type": "web", "url": "https://habr.com/ru/articles/881502/"}
    ]
    # Обработка всех источников
    processor.process_all_sources(sources)

# Соц. сети

In [3]:
from googleapiclient.discovery import build
import pandas as pd
from abc import ABC, abstractmethod
import time
import logging
import re
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import nltk

# Скачиваем необходимые ресурсы NLTK (если они еще не скачаны)
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt_tab')

# Configure logging
logging.basicConfig(level=logging.INFO)

class DataSource(ABC):
    """
    Абстрактный базовый класс для работы с источниками данных.
    """
    def __init__(self, source_id: str):
        self.source_id = source_id

    @abstractmethod
    def fetch_data(self) -> list:
        """
        Метод для получения данных из источника.
        :return: Список словарей с данными.
        """
        pass

    def process_data(self, data: list) -> list:
        """
        Метод для обработки и очистки данных.
        :param data: Список словарей с данными.
        :return: Обработанный список словарей с данными.
        """
        processed_data = []
        for item in data:
            processed_item = item.copy()
            # Очищаем текст комментария, если он существует
            if 'text' in processed_item and isinstance(processed_item['text'], str):
                processed_item['text'] = self.clean_text(processed_item['text'])
            elif 'text' in processed_item:
                # Если текст отсутствует или является NaN, заменяем его на пустую строку
                processed_item['text'] = ''
            processed_data.append(processed_item)
        return processed_data

    def clean_text(self, text: str) -> str:
        """
        Метод для очистки текста.
        :param text: Исходный текст.
        :return: Очищенный текст.
        """
        if not isinstance(text, str):  # Проверка, чтобы убедиться, что это строка
            return ''

        # Удаление HTML тегов
        text = re.sub(r'<.*?>', '', text)
        # Удаление упоминаний (@username)
        text = re.sub(r'@\w+', '', text)
        # Удаление URL
        text = re.sub(r'http\S+|www.\S+', '', text)
        # Удаление специальных символов и цифр
        text = re.sub(r'[^a-zA-Z\s]', '', text)
        # Приведение к нижнему регистру
        text = text.lower()
        # Токенизация
        tokens = word_tokenize(text)
        # Удаление стоп-слов
        stop_words = set(stopwords.words('english'))
        tokens = [word for word in tokens if word not in stop_words]
        # Лемматизация
        lemmatizer = WordNetLemmatizer()
        tokens = [lemmatizer.lemmatize(word) for word in tokens]
        # Соединение слов обратно в строку
        cleaned_text = ' '.join(tokens)
        return cleaned_text

    def to_dataframe(self, data: list) -> pd.DataFrame:
        """
        Преобразует данные в DataFrame.
        :param data: Список словарей с данными.
        :return: DataFrame.
        """
        return pd.DataFrame(data)

    def save_to_csv(self, data: list, output_file: str):
        """
        Сохраняет данные в CSV файл.
        :param data: Список словарей с данными.
        :param output_file: Имя выходного файла.
        """
        df = self.to_dataframe(data)
        df.to_csv(output_file, index=False, encoding="utf-8")
        logging.info(f"Data saved to {output_file}")

class YouTubeDataSource(DataSource):
    """
    Класс для работы с комментариями YouTube.
    """
    def __init__(self, video_id: str, api_key: str):
        super().__init__(source_id=video_id)
        self.api_key = api_key
        self.youtube = build('youtube', 'v3', developerKey=self.api_key)

    def fetch_data(self) -> list:
        """
        Получает комментарии для указанного видео.
        :return: Список словарей с комментариями.
        """
        comments = []
        next_page_token = None

        while True:
            try:
                request = self.youtube.commentThreads().list(
                    part="snippet",
                    videoId=self.source_id,
                    maxResults=100,
                    pageToken=next_page_token
                )
                response = request.execute()

                for item in response.get("items", []):
                    comment = {
                        "comment_id": item["id"],
                        "text": item["snippet"]["topLevelComment"]["snippet"]["textDisplay"],
                        "author": item["snippet"]["topLevelComment"]["snippet"]["authorDisplayName"],
                        "published_at": item["snippet"]["topLevelComment"]["snippet"]["publishedAt"],
                        "like_count": item["snippet"]["topLevelComment"]["snippet"]["likeCount"]
                    }
                    comments.append(comment)

                next_page_token = response.get("nextPageToken")
                if not next_page_token:
                    break

                # Add a delay to avoid hitting rate limits
                time.sleep(1)

            except Exception as e:
                logging.error(f"An error occurred: {e}")
                break

        logging.info(f"Collected {len(comments)} comments for video ID: {self.source_id}")
        return comments


class CSVDataSource(DataSource):
    """
    Класс для работы с CSV файлами.
    """
    def __init__(self, file_path: str):
        super().__init__(source_id=file_path)

    def fetch_data(self) -> list:
        """
        Читает данные из CSV файла.
        :return: Список словарей с данными.
        """
        try:
            df = pd.read_csv(self.source_id)
            data = df.to_dict(orient='records')
            logging.info(f"Loaded {len(data)} records from {self.source_id}")
            return data
        except Exception as e:
            logging.error(f"Error loading CSV file: {e}")
            return []


# Пример использования
if __name__ == "__main__":
    # Константы для YouTube
    API_KEY = ''
    VIDEO_ID = 'AVYfyTvc9KY'

    # Создаем экземпляр YouTubeDataSource
    youtube_source = YouTubeDataSource(video_id=VIDEO_ID, api_key=API_KEY)
    youtube_comments = youtube_source.fetch_data()

    # Обрабатываем данные
    processed_youtube_comments = youtube_source.process_data(youtube_comments)

    # Сохраняем данные в CSV
    youtube_output_file = f"youtube_comments_{VIDEO_ID}.csv"
    youtube_source.save_to_csv(processed_youtube_comments, youtube_output_file)

    # Создаем экземпляр CSVDataSource
    csv_file_path = youtube_output_file  # Можно использовать любой другой CSV файл
    csv_source = CSVDataSource(file_path=csv_file_path)
    csv_data = csv_source.fetch_data()

    # Обрабатываем данные из CSV
    processed_csv_data = csv_source.process_data(csv_data)

    # Сохраняем данные из CSV в новый файл (для демонстрации)
    csv_output_file = f"processed_data_{VIDEO_ID}.csv"
    csv_source.save_to_csv(processed_csv_data, csv_output_file)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
